# 10章　特徴量選択による次元削減 
## レシピ10.1　数値特徴量の分散による閾値処理 


In [ ]:
# ライブラリをロード
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

# テスト用のデータをロード
iris = datasets.load_iris()

# 特徴量とターゲットを作成
features = iris.data
target = iris.target

# 閾値を作成
thresholder = VarianceThreshold(threshold=.5)

# 分散の大きい特徴量行列を作成
features_high_variance = thresholder.fit_transform(features)

# 分散の大きい特徴量行列を表示
features_high_variance[0:3]

In [ ]:
# 分散を表示
thresholder.fit(features).variances_

In [ ]:
# ライブラリをロード
from sklearn.preprocessing import StandardScaler

# 特徴量行列を標準化
scaler = StandardScaler()
features_std = scaler.fit_transform(features)

# それぞれの特徴量を算出
selector = VarianceThreshold()
selector.fit(features_std).variances_

## レシピ10.2　2値特徴量の分散閾値処理 


In [ ]:
# ライブラリをロード
from sklearn.feature_selection import VarianceThreshold

# 特徴量行列を下記のように作成:
# 特徴量 0: 80% クラス 0
# 特徴量 1: 80% クラス 1
# 特徴量 2: 60% クラス 0, 40% クラス 1
features = [[0, 1, 0],
            [0, 1, 1],
            [0, 1, 0],
            [0, 1, 1],
            [1, 0, 0]]

# 分散で閾値処理
thresholder = VarianceThreshold(threshold=(.75 * (1 - .75)))
thresholder.fit_transform(features)

## レシピ10.3　強く相関した特徴量の取り扱い 


In [ ]:
# ライブラリをロード
import pandas as pd
import numpy as np

# 2つの特徴量が強く相関した特徴量行列を作成
features = np.array([[1, 1, 1],
                     [2, 2, 0],
                     [3, 3, 1],
                     [4, 4, 0],
                     [5, 5, 1],
                     [6, 6, 0],
                     [7, 7, 1],
                     [8, 7, 0],
                     [9, 7, 1]])

# 特徴量行列をDataFrameに変換
dataframe = pd.DataFrame(features)

# 相関行列を作成
corr_matrix = dataframe.corr().abs()

# 相関行列の上三角を選択
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# 相関が0.95以上になる特徴量列のインデックスを抽出
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# 特徴量を削除
dataframe.drop(dataframe.columns[to_drop], axis=1).head(3)

In [ ]:
# 相関行列を表示
dataframe.corr()

In [ ]:
# 相関行列の上三角行列を表示
upper

## レシピ10.4　クラス分類に無関係な特徴量の削除 


In [ ]:
# ライブラリをロード
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

# データをロード
iris = load_iris()
features = iris.data
target = iris.target

# データを整数に変換して、カテゴリデータとして扱う
features = features.astype(int)

# カイ二乗統計量が最大の2つの特徴量を選択
chi2_selector = SelectKBest(chi2, k=2)
features_kbest = chi2_selector.fit_transform(features, target)

# 結果を表示
print("もとの特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_kbest.shape[1])

In [ ]:
# 最も高いF値を持つ特徴量を2つ選択
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

# 結果を表示
print("もとの特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_kbest.shape[1])

In [ ]:
# ライブラリをロード
from sklearn.feature_selection import SelectPercentile

# F値が上位75パーセントの特徴量を選択
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)

# 結果を表示
print("もとの特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_kbest.shape[1])

## レシピ10.5　再帰的な特徴量の除去 


In [ ]:
# ライブラリをロード
import warnings
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

# わずらわしいが無害な警告を抑止
warnings.filterwarnings(action="ignore", module="scipy",
message="^internal gelsd")

# 特徴量行列、ターゲットベクトル、真の相関係数を作成
features, target = make_regression(n_samples = 10000,
                                   n_features = 100,
                                   n_informative = 2,
                                   random_state = 1)

# 線形回帰器を作成
ols = linear_model.LinearRegression()

# 再帰的に特徴量を除去
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
rfecv.transform(features)

In [ ]:
# 維持すべき特徴量の数
rfecv.n_features_

In [ ]:
# 維持すべき特徴量を表示
rfecv.support_

In [ ]:
# 特徴量のランクを表示（最良が1）
rfecv.ranking_